In [ ]:
!pip install cantera

from tqdm import tqdm
import cantera as ct
import pandas as pd
import time

In [ ]:
# Define the conditions for the simulations
transport_models = ["multicomponent", "mixture-averaged", "unity-Lewis-number"]
T = 300
p = ct.one_atm
fuel = {"H2": 1}
oxidizer = {"O2": 1, "N2": 3.76}
equivalence_ratios = [0.5, 1.0, 2.0]

# Create a pandas dataframe to store the results
df_stats = pd.DataFrame(columns=["transport_model", "time", "mechanism"])

In [ ]:
#
# Study of time vs mechanism: h2o2
#
n_simulations = 20
times_per_mechanism = {}
mech = "h2o2.yaml"
for transport_model in transport_models:
    print(f"Running {n_simulations} simulations for mechanism {mech}")
    times = []
    for i in tqdm(range(n_simulations), desc=f"Mechanism {mech}"):
        delta_t = 0
        for equivalence_ratio in equivalence_ratios:
            gas = ct.Solution(mech)
            gas.TPX = T, p, fuel
            gas.set_equivalence_ratio(equivalence_ratio, fuel, oxidizer)
            flame = ct.FreeFlame(gas, width=0.3)
            flame.transport_model = transport_model

            flame.energy_enabled = True
            flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

            flame.set_max_jac_age(10, 10)
            flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
            flame.max_time_step_count = 5000    

            # Set steady-state tolerance to relative and absolute tolerances of 1e-15
            flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
            flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])

            start = time.time()
            flame.solve(loglevel=0, auto=True)
            end = time.time()
            delta_t += end - start
        times.append(delta_t)
    times_per_mechanism[mech] = times
    avg_time = sum(times)/n_simulations
    print(f"Transport {transport_model} took {avg_time:.2f} seconds on average")
    df_stats.loc[len(df_stats)] = [transport_model, avg_time, mech]

In [ ]:
#
# Study of time vs mechanism: gri30
# 
n_simulations = 20
times_per_mechanism = {}
mech = "gri30.yaml"
for transport_model in transport_models:
    print(f"Running {n_simulations} simulations for mechanism {mech}")
    times = []
    for i in tqdm(range(n_simulations), desc=f"Mechanism {mech}"):
        delta_t = 0
        for equivalence_ratio in equivalence_ratios:
            gas = ct.Solution(mech)
            gas.TPX = T, p, fuel
            gas.set_equivalence_ratio(equivalence_ratio, fuel, oxidizer)
            flame = ct.FreeFlame(gas, width=0.3)
            flame.transport_model = transport_model

            flame.energy_enabled = True
            flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

            flame.set_max_jac_age(10, 10)
            flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
            flame.max_time_step_count = 5000    

            # Set steady-state tolerance to relative and absolute tolerances of 1e-15
            flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
            flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])

            start = time.time()
            flame.solve(loglevel=0, auto=True)
            end = time.time()
            delta_t += end - start
        times.append(delta_t)
    times_per_mechanism[mech] = times
    avg_time = sum(times)/n_simulations
    print(f"Transport {transport_model} took {avg_time:.2f} seconds on average")
    df_stats.loc[len(df_stats)] = [transport_model, avg_time, mech]